In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk


# Compute scoremaps for linearly normalized version

In [18]:
stack = 'MD657'

for classifier_id in range(83, 85):
    t = time.time()
    sys.stderr.write('running svm classifier ...')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

    t = time.time()
    sys.stderr.write('Resampling scoremaps ...')

    # downscale = 8
    downscale = 32

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(classifier_id)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v4.py'),
                    'stack': stack,
                    'classifier_id': classifier_id,
                    'downscale': downscale},
    #                 kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[metadata_cache['sections_to_filenames'][stack][150]]),
                    kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
                               if fn.split('-')[1][0] == 'F']),
                    argument_type='list2'
                   )


    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 277s

running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 317.484612 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 70.839120 seconds
running svm classifier ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 322.478262 seconds
Resampling scoremaps ...Child returned 0
16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 65.922713 seconds


# Regular version - different stacks

In [11]:
detector_id = 15

In [ ]:
for stack in [
# 'MD585',
 'MD589',
 'MD590',
 'MD591',
 'MD592',
 'MD593',
 'MD594',
 'MD595',
 'MD598',
 'MD599',
 'MD602',
 'MD603',
]:
    
    t = time.time()
    sys.stderr.write('running svm classifier ...')

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v5.py'),
                    'stack': stack,
                    'detector_id': detector_id},
                    kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
    #                                             if fn.split('-')[1][0] == 'F']),
    #                 kwargs_list=dict(filenames=q),
                    argument_type='list2')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

    # 16 m4.4xlarge, 411s/140s
    
    ########################################################
    
    # Feature computation window spacing is 56, so the equivalent downscale factor is 56. Still larger than 32.
    # It does not seems necessary to generate down8 score maps because the extra data is interpolated anyway.
    t = time.time()
    sys.stderr.write('Resampling scoremaps ...')

    # downsample = 8
    downsample = 32

    run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d %(downsample)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v5.py'),
                    'stack': stack,
                    'detector_id': detector_id,
                    'downsample': downsample},
                    kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
    #                            if fn.split('-')[1][0] == 'F']),
                    argument_type='list2'
                   )


    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    # down32, 16 m4.4xlarge, 60s
    # down8, 16 m4.4xlarge, 246s
    
    ##############################################################
    
    # Must generate down32 scoremap first, 
    # because even if viz_downsample=8 the scoermaps the script trying to load are still down32.

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_label_text = False
    viz_downsample = 8
    # viz_downsample = 32
    cmap = 'jet'
#     background_image_version = 'grayDefaultJpeg'
    background_image_version = 'grayJpeg'
    # background_image_version = 'contrastStretched'

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d %(downsample)d --cmap %(cmap)s %(add_label_text)s -b %(bg_img_ver)s' % \
                    {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v5.py'),
                    'stack': stack,
                     'detector_id': detector_id,
                    'downsample': viz_downsample,
                     'cmap': cmap,
                    'add_label_text': '-a' if add_label_text else '',
                    'bg_img_ver': background_image_version},
                    kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
    #                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
    #                            if fn.split('-')[1][0] == 'F']),
                    argument_type='list2'
                   )

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    # For one stack
    # 16 nodes, downsample 32, 75s
    # 8 nodes, downscale 8, 6000s
    # 8 nodes, downscale 8, scratch/ 1510s
    # 16 m4.4xlarge, downscale 8, scratch/ 700s
    
    ####################################################################################
    
    
    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    use_nissl_only = False
    volume_downscale = 32

    run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(detector_id)d %(downscale)d %(use_nissl_only)s' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v5.py'),
                    'stack': stack,
                    'detector_id': detector_id,
                    'downscale': volume_downscale,
                    'use_nissl_only': '-n' if use_nissl_only else ''},
                    kwargs_list=dict(structure=all_known_structures),
    #                 kwargs_list=dict(structure=['VLL']),
                    argument_type='single')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    # 16 m4.4xlarge, downscale 32, 55s
    
    ####################################################################################
    
    t = time.time()
    sys.stderr.write('compute score volume gradients...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(detector_id)d %(downscale)d' % \
                    {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v5.py'),
                    'stack': stack,
                    'detector_id': detector_id,
                    'downscale': volume_downscale},
                    kwargs_list=dict(structure=all_known_structures),
    #                 kwargs_list=dict(structure=['VLL']),
                    argument_type='single')

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

    # 55 seconds (16 nodes write to respective local /scratch)
    # More than 1 simul. processes are not beneficial as they cause local write contention.
    # 156 seconds (16 nodes simul. write to /shared)
    # 310 seconds (single node, sequential write to /shared)
    # 16 m4.4xlarge, downscale 32, 55s

running svm classifier ...5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 391.731642 seconds
Resampling scoremaps ...5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 125.535488 seconds
visualize scoremaps ...5 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


# Regular version - one stack

In [14]:
# classifier_id = 38
stack = 'MD658'
detector_id = 13

In [15]:
# m = check_output("aws s3 ls s3://mousebrainatlas-data/CSHL_patch_scores/MD589/".split()).split('\n')
# q = list(set(metadata_cache['valid_filenames'][stack]) - set([f.strip()[4:-12] for f in m]))

In [3]:
request_compute_nodes(16, 'yuncong')

Setting autoscaling group cfncluster-yuncongcluster-ComputeFleet-15GRGL8BMOLW2 capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


In [6]:
get_num_nodes()

16

In [18]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v5.py'),
                'stack': stack,
                'detector_id': detector_id},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
#                                             if fn.split('-')[1][0] == 'F']),
#                 kwargs_list=dict(filenames=q),
                argument_type='list2')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 16 m4.4xlarge, 411s/140s

running svm classifier ...16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 140.688367 seconds


In [19]:
# Feature computation window spacing is 56, so the equivalent downscale factor is 56. Still larger than 32.
# It does not seems necessary to generate down8 score maps because the extra data is interpolated anyway.

t = time.time()
sys.stderr.write('Resampling scoremaps ...')

# downsample = 8
downsample = 32

run_distributed(command='%(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d %(downsample)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v5.py'),
                'stack': stack,
                'detector_id': detector_id,
                'downsample': downsample},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
#                            if fn.split('-')[1][0] == 'F']),
                argument_type='list2'
               )


wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# down32, 16 m4.4xlarge, 60s
# down8, 16 m4.4xlarge, 246s

Resampling scoremaps ...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 55.400345 seconds


In [20]:
# Must generate down32 scoremap first, 
# because even if viz_downsample=8 the scoermaps the script trying to load are still down32.

# This requires having grayJpeg output available first.

t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
viz_downsample = 8
# viz_downsample = 32
cmap = 'jet'
background_image_version = 'grayDefaultJpeg'
# background_image_version = 'grayJpeg'
# background_image_version = 'contrastStretched'

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\' %(detector_id)d %(downsample)d --cmap %(cmap)s %(add_label_text)s -b %(bg_img_ver)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v5.py'),
                'stack': stack,
                 'detector_id': detector_id,
                'downsample': viz_downsample,
                 'cmap': cmap,
                'add_label_text': '-a' if add_label_text else '',
                'bg_img_ver': background_image_version},
                kwargs_list=dict(filenames=metadata_cache['valid_filenames'][stack]),
#                 kwargs_list=dict(filenames=[fn for fn in metadata_cache['valid_filenames'][stack]
#                            if fn.split('-')[1][0] == 'F']),
                argument_type='list2'
               )

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# For one stack
# 16 nodes, downsample 32, 75s
# 8 nodes, downscale 8, 6000s
# 8 nodes, downscale 8, scratch/ 1510s
# 16 m4.4xlarge, downscale 8, scratch/ 700s

visualize scoremaps ...16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


rm -f ~/stderr_*; rm -f ~/stdout_*


qsub returned.
done in 446.751777 seconds


## Construct score volumes

In [6]:
stack = 'MD589'
detector_id = 13

In [28]:
request_compute_nodes(0, 'yuncong')

Setting autoscaling group cfncluster-yuncongcluster-ComputeFleet-15GRGL8BMOLW2 capaticy to 0...it may take more than 5 minutes for SGE to know new hosts.


In [30]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

use_nissl_only = False
volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(detector_id)d %(downscale)d %(use_nissl_only)s' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v5.py'),
                'stack': stack,
                'detector_id': detector_id,
                'downscale': volume_downscale,
                'use_nissl_only': '-n' if use_nissl_only else ''},
                kwargs_list=dict(structure=all_known_structures),
#                 kwargs_list=dict(structure=['VLL']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# 16 m4.4xlarge, downscale 32, 55s

constructing score volumes ...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 55.890324 seconds


In [31]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s %%(structure)s %(detector_id)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v5.py'),
                'stack': stack,
                'detector_id': detector_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
#                 kwargs_list=dict(structure=['VLL']),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

# 55 seconds (16 nodes write to respective local /scratch)
# More than 1 simul. processes are not beneficial as they cause local write contention.
# 156 seconds (16 nodes simul. write to /shared)
# 310 seconds (single node, sequential write to /shared)
# 16 m4.4xlarge, downscale 32, 55s

compute score volume gradients...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 50.544045 seconds


# Global alignment

In [2]:
# classifier_setting = 38
detector_id = 15
warp_setting = 16

In [3]:
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 'VLL_L', 'VLL_R', '7n_L']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCP_R']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L','5N_R','Pn_L', 'Pn_R', 'SNR_L', 'SNR_R', 
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCP_R', 'IC', 'SC']
# structure_subset = ['7N_L', '7N_R', '12N', '5N_L', '5N_R', 'SNR_L', 'SNR_R', 'Pn_L', 'Pn_R',
#                     'VLL_L', 'VLL_R', '7n_L', '7n_R', 'Tz_L', 'Tz_R', 
#                     'VCA_L', 'VCA_R', 'VCP_L', 'VCP_R']
structure_subset = all_known_structures_sided

In [4]:
atlas_name = 'atlasV5'

In [6]:
# request_compute_nodes(5, 'yuncong')

In [5]:
stacks = [
#  'MD589',
#  'MD590',
#  'MD591',
#  'MD592',
#  'MD593',
#  'MD594',
#  'MD595',
#  'MD598',
#  'MD599',
 'MD602',
 'MD603']

In [ ]:
# Align - Sequential

# m4.4xlarge is required.

for stack in stacks:
    t = time.time()
    sys.stderr.write('align subject to atlas ...')

    execute_command('%(script_path)s %(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --trial_num 1 --structures \'%(structures)s\'' % \
        {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v4.py'),
         'atlas_name': atlas_name,
        'warp_setting': 20,
        'detector_id': detector_id,
        'structures': json.dumps(structure_subset),
        'stack': stack})

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...

/shared/MouseBrainAtlas/registration/global_registration_v4.py MD589 atlasV5 20 15 --trial_num 1 --structures '["5N_L", "5N_R", "6N_L", "6N_R", "7N_L", "7N_R", "7n_L", "7n_R", "Amb_L", "Amb_R", "LC_L", "LC_R", "LRt_L", "LRt_R", "Pn_L", "Pn_R", "Tz_L", "Tz_R", "VLL_L", "VLL_R", "RMC_L", "RMC_R", "SNC_L", "SNC_R", "SNR_L", "SNR_R", "3N_L", "3N_R", "4N_L", "4N_R", "Sp5I_L", "Sp5I_R", "Sp5O_L", "Sp5O_R", "Sp5C_L", "Sp5C_R", "PBG_L", "PBG_R", "10N_L", "10N_R", "VCA_L", "VCA_R", "VCP_L", "VCP_R", "DC_L", "DC_R", "AP", "12N", "RtTg", "sp5", "outerContour", "SC", "IC"]'
/shared/MouseBrainAtlas/registration/global_registration_v4.py MD590 atlasV5 20 15 --trial_num 1 --structures '["5N_L", "5N_R", "6N_L", "6N_R", "7N_L", "7N_R", "7n_L", "7n_R", "Amb_L", "Amb_R", "LC_L", "LC_R", "LRt_L", "LRt_R", "Pn_L", "Pn_R", "Tz_L", "Tz_R", "VLL_L", "VLL_R", "RMC_L", "RMC_R", "SNC_L", "SNC_R", "SNR_L", "SNR_R", "3N_L", "3N_R", "4N_L", "4N_R", "Sp5I_L", "Sp5I_R", "Sp5O_L", "Sp5O_R", "Sp5C_L", "Sp5C_R", "PBG_L"

done in 23765.596945 seconds


In [8]:
# Align - Sequential

# m4.4xlarge is required.

for stack in stacks:
    t = time.time()
    sys.stderr.write('align all subjects to atlas ...')

    execute_command('%(script_path)s %(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --trial_num 1 --structures \'%(structures)s\'' % \
        {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v4.py'),
         'atlas_name': atlas_name,
        'warp_setting': 16,
        'detector_id': detector_id,
        'structures': json.dumps(structure_subset),
        'stack': stack})
    
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

align all subjects to atlas ...

/shared/MouseBrainAtlas/registration/global_registration_v4.py MD602 atlasV5 16 15 --trial_num 1 --structures '["5N_L", "5N_R", "6N_L", "6N_R", "7N_L", "7N_R", "7n_L", "7n_R", "Amb_L", "Amb_R", "LC_L", "LC_R", "LRt_L", "LRt_R", "Pn_L", "Pn_R", "Tz_L", "Tz_R", "VLL_L", "VLL_R", "RMC_L", "RMC_R", "SNC_L", "SNC_R", "SNR_L", "SNR_R", "3N_L", "3N_R", "4N_L", "4N_R", "Sp5I_L", "Sp5I_R", "Sp5O_L", "Sp5O_R", "Sp5C_L", "Sp5C_R", "PBG_L", "PBG_R", "10N_L", "10N_R", "VCA_L", "VCA_R", "VCP_L", "VCP_R", "DC_L", "DC_R", "AP", "12N", "RtTg", "sp5", "outerContour", "SC", "IC"]'


done in 18822.409016 seconds
align all subjects to atlas ...

/shared/MouseBrainAtlas/registration/global_registration_v4.py MD603 atlasV5 16 15 --trial_num 1 --structures '["5N_L", "5N_R", "6N_L", "6N_R", "7N_L", "7N_R", "7n_L", "7n_R", "Amb_L", "Amb_R", "LC_L", "LC_R", "LRt_L", "LRt_R", "Pn_L", "Pn_R", "Tz_L", "Tz_R", "VLL_L", "VLL_R", "RMC_L", "RMC_R", "SNC_L", "SNC_R", "SNR_L", "SNR_R", "3N_L", "3N_R", "4N_L", "4N_R", "Sp5I_L", "Sp5I_R", "Sp5O_L", "Sp5O_R", "Sp5C_L", "Sp5C_R", "PBG_L", "PBG_R", "10N_L", "10N_R", "VCA_L", "VCA_R", "VCP_L", "VCP_R", "DC_L", "DC_R", "AP", "12N", "RtTg", "sp5", "outerContour", "SC", "IC"]'


KeyboardInterrupt: 

In [ ]:
# Align

# m4.4xlarge is required.

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --trial_num 5 --structures \'%(structures)s\'' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'global_registration_v4.py'),
                 'atlas_name': atlas_name,
                'warp_setting': warp_setting,
                'detector_id': detector_id,
                'structures': json.dumps(structure_subset)},
                 kwargs_list=dict(stack=stacks),
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 372s (1 trial); 750s (5 trials)

In [23]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'atlas_name': atlas_name,
                'detector_id': detector_id,
                 'warp_setting': warp_setting
                },
                kwargs_list={'stack': stacks},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
# For one stack, 60 seconds

transform atlas ...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 45.368512 seconds


In [40]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

# bg_img_version = 'grayDefaultJpeg'
bg_img_version = 'grayJpeg'

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --out_downsample 8 -v %(bg_img_version)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v5.py'),
                'atlas_name': atlas_name,
                'detector_id': detector_id,
                 'warp_setting': warp_setting,
                 'bg_img_version': bg_img_version
                },
                kwargs_list={'stack': stacks},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))
# 700s

visualize aligned atlas overlay ...16 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 111.255946 seconds


# Local align

In [4]:
detector_id = 15
warp_setting = 17

In [5]:
atlas_name = 'atlasV5'

In [6]:
stacks = ['MD585',
 'MD589',
 'MD590',
 'MD591',
 'MD592',
 'MD593',
 'MD594',
 'MD595',
 'MD598',
 'MD599',
 'MD602',
 'MD603']

In [44]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

# stacks = ['MD642', 'MD652', 'MD653', 'MD657', 'MD658']

kwargs_list = \
[dict(stack=stack, structures=shell_escape(json.dumps(all_known_structures_sided[f:l+1])))
for f,l in first_last_tuples_distribute_over(0, len(all_known_structures_sided)-1, get_num_nodes())
for stack in stacks]

# kwargs_list = \
# [dict(stack='MD661', structures=shell_escape(json.dumps(['5N_L']))),
#  dict(stack='MD661', structures=shell_escape(json.dumps(['7N_L']))),
# dict(stack='MD662', structures=shell_escape(json.dumps(['5N_L']))),
# dict(stack='MD662', structures=shell_escape(json.dumps(['7N_L'])))]

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --trial_num 5 -s %%(structures)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'local_registration_v4.py'),
                'detector_id': detector_id,
                 'warp_setting': warp_setting,
                'atlas_name': atlas_name},
                 kwargs_list=kwargs_list,
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 2047s for 5 trials.
# IC and SC are super slow...
# 58488 s

LOCAL align ...

rm -f ~/stderr_*; rm -f ~/stdout_*


16 nodes available.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 58488.887352 seconds


In [9]:
request_compute_nodes(12, 'yuncong')

Setting autoscaling group cfncluster-yuncongcluster-ComputeFleet-1R5OSQE66IOTP capaticy to 12...it may take more than 5 minutes for SGE to know new hosts.


In [13]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'transform_brains_v4.py'),
                'detector_id': detector_id,
                 'warp_setting': warp_setting,
                'atlas_name': atlas_name
                },
                kwargs_list={'stack': stacks},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 60 seconds

transform atlas ...12 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 50.600226 seconds


In [14]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

# bg_img_version = 'grayDefaultJpeg'
bg_img_version = 'grayJpeg'

run_distributed(command='%(script_path)s %%(stack)s %(atlas_name)s %(warp_setting)d %(detector_id)d --out_downsample 8 -v %(bg_img_version)s' % \
                {'script_path': os.path.join(REPO_DIR, 'registration', 'visualize_registration_v5.py'),
                'detector_id': detector_id,
                 'warp_setting': warp_setting,
                 'bg_img_version': bg_img_version,
                'atlas_name': atlas_name
                },
                kwargs_list={'stack': stacks},
                argument_type='single')

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # For one stack, 110 seconds

visualize aligned atlas overlay ...12 nodes available.


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.
qsub returned.
done in 110.789422 seconds


# Transform locally transformed volumes back to atlas space

In [ ]:
classifier_setting = 2
warp_setting = 3

In [ ]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

# Measure Global Tx Confidence

In [ ]:
warp_setting = 1

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

# Measure Local Tx Confidence

In [ ]:
warp_setting = 5
classifier_setting = 2

In [ ]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins